# RENTING BARCELONA

### IMPORTS

In [801]:
import pandas as pd
import numpy as np
import re

## RENT DATASET

In [802]:
# Import csv de rent. 
rent = pd.read_csv("/Users/roraimachavez/Downloads/7.IRONHACK/Mini projects/sql/csv/Barcelona_rent_price.csv")

In [803]:
rent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Year           4622 non-null   int64  
 1   Trimester      4622 non-null   int64  
 2   District       4622 non-null   object 
 3   Neighbourhood  4622 non-null   object 
 4   Average _rent  4622 non-null   object 
 5   Price          4622 non-null   float64
dtypes: float64(1), int64(2), object(3)
memory usage: 216.8+ KB


In [804]:
# Column name in lower cap.
rent.columns = rent.columns.str.lower()

In [805]:
# Filter per the year with want.
rent_filter = rent.loc[(rent['year']== 2022)] 

# Assign the new dataframe to the old one
rent = rent_filter.copy() 

In [806]:
# Reset index. 
rent = rent.reset_index(drop=True)

In [807]:
rent.head()

,year,trimester,district,neighbourhood,average _rent,price
0,2022,1,Ciutat Vella,el Raval,average rent (euro/month),917.0
1,2022,1,Ciutat Vella,Gothic Quarter,average rent (euro/month),1181.6
2,2022,1,Ciutat Vella,la Barceloneta,average rent (euro/month),999.8
3,2022,1,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera",average rent (euro/month),1022.8
4,2022,1,Eixample,Fort Pienc,average rent (euro/month),957.9


### Get the DISTRICT DATAFRAME. 

Here we need to make a DataFrame to take district id. So we:

- Get unique values.
- Generated the id with a index.
- Save it. 

In [808]:
# Make a variable for the uniques categories in district.
unique_district =rent["district"].unique()

In [809]:
# Them, make a DataFrame of this categories.
district = pd.DataFrame(unique_district, columns=['district'])

# Them make a index for this column. 
district["district_id"] = pd.Series(district["district"], dtype="category").cat.codes + 1

In [810]:
district.reset_index(drop=True, inplace=True)

In [811]:
district

,district,district_id
0,Ciutat Vella,1
1,Eixample,2
2,Sants-Montjuic,9
3,Les Corts,5
4,Sarria-Sant Gervasi,10
5,Gracia,3
6,Horta-Guinardo,4
7,Nou Barris,6
8,Sant Andreu,7
9,Sant Marti,8


In [812]:
# Save the DataFrame.
# district.to_csv('district.csv', index=True)

### Get the NEIGHBOURHOOD DATAFRAME. 

Here we need to make a DataFrame to take neighbourhood id. So we:

- Get unique values.
- Generated the id with a index.
- Merge to distric id because we need a table of neighbourhood with the district id. 
- Save it.

In [813]:
# Make a variable for the uniques categories in district.
unique_neighbourhood =rent["neighbourhood"].unique()

In [814]:
# Them, make a DataFrame of this categories.
rent_neighbourhood = pd.DataFrame(unique_neighbourhood, columns=["neighbourhood"])

# Them make a index for this column. 
rent_neighbourhood["neighbourhood_id"] = pd.Series(rent_neighbourhood["neighbourhood"], dtype="category").cat.codes + 1

**HERE WE CAME BACK TO RENT FIRTS TO GET EVERYTHING TOGETHER**

In [815]:
# Merge distric df firts. 
rent = pd.merge(rent, district, on='district')

In [816]:
# Merge neighbourhood df. 
rent = pd.merge(rent, rent_neighbourhood, on='neighbourhood')

In [817]:
rent.head()

,year,trimester,district,neighbourhood,average _rent,price,district_id,neighbourhood_id
0,2022,1,Ciutat Vella,el Raval,average rent (euro/month),917.0,1,43
1,2022,1,Ciutat Vella,el Raval,average rent per surface (euro/m2),15.5,1,43
2,2022,2,Ciutat Vella,el Raval,average rent (euro/month),872.0,1,43
3,2022,2,Ciutat Vella,el Raval,average rent per surface (euro/m2),14.7,1,43
4,2022,1,Ciutat Vella,Gothic Quarter,average rent (euro/month),1181.6,1,6


**FINISH MERGE RENT**

In [818]:
# We make a new df dropping columns with don't need from rent df and we can make the new one. We could merge district df with the neighbourhood one but we decide to make it like this. 
neighbourhood = rent.loc[:, [ "district_id", "neighbourhood", "neighbourhood_id"]]

In [819]:
neighbourhood.head()

,district_id,neighbourhood,neighbourhood_id
0,1,el Raval,43
1,1,el Raval,43
2,1,el Raval,43
3,1,el Raval,43
4,1,Gothic Quarter,6


In [820]:
# We delete duplicates rows since the loc generated many.
neighbourhood.drop_duplicates(inplace=True)

In [821]:
neighbourhood.head()

,district_id,neighbourhood,neighbourhood_id
0,1,el Raval,43
4,1,Gothic Quarter,6
8,1,la Barceloneta,46
12,1,"Sant Pere, Santa Caterina i la Ribera",21
16,2,Fort Pienc,5


In [822]:
# Save neighbourhood df. 
# neighbourhood.to_csv('neighbourhood.csv', index=True)

### CLEAN RENT DATAFRAME

In [823]:
# Delete columns we don't want.
rent = rent.drop(["district", "neighbourhood", "trimester", "year" ], axis=1)

# Drop cateogry we don't want. 
rent = rent[rent['average _rent'] != 'average rent per surface (euro/m2)']

In [824]:
rent.head()

,average _rent,price,district_id,neighbourhood_id
0,average rent (euro/month),917.0,1,43
2,average rent (euro/month),872.0,1,43
4,average rent (euro/month),1181.6,1,6
6,average rent (euro/month),1062.5,1,6
8,average rent (euro/month),999.8,1,46


In [825]:
rent.shape

(135, 4)

In [826]:
# Group the columns we need making the calculation of the mean. 
rent_group = rent.groupby(['neighbourhood_id', 'district_id'])['price'].mean()

# Make a new DataFrame.
rent = pd.DataFrame(rent_group)

In [827]:
rent.head()

,,price
neighbourhood_id,district_id,
1,4,775.00
2,6,753.15
3,6,547.50
4,8,1101.70
5,2,1032.40


In [828]:
# Generated a random index. 
rent['rent_id'] = np.random.randint(0, 1000, size=len(rent))

In [829]:
# Change column name. 
rent.rename(columns = {'price': 'price_avg'}, inplace=True)

In [830]:
rent

,,price_avg,rent_id
neighbourhood_id,district_id,,
1,4,775.00,644
2,6,753.15,644
3,6,547.50,448
4,8,1101.70,795
5,2,1032.40,718
...,...,...,...
64,8,1373.80,157
65,3,984.30,970
66,5,1031.40,711


In [831]:
# Save rent df. 
# rent.to_csv('rent.csv', index=True)

## AIRBNB DATASET

In [975]:
# Import csv. 
airbnb = pd.read_csv("/Users/roraimachavez/Downloads/7.IRONHACK/Mini projects/sql/csv/barcelona_airbnb.csv")

In [976]:
# Convert lower case. 
airbnb.columns = airbnb.columns.str.lower()

In [977]:
airbnb.head()

,tipo,nombrealojamiento,zona,nhab,servicios,fechas,valoracion,evaluaciones,noche,total
0,Habitación privada,"Habitacion centro Barcelona, Ciutat Vella room.",Ciutat Vella,2 huéspedes · 1 dormitorio · 1 cama · 1 baño c...,Wifi · Cocina,17 dic. – 19 dic.,"4,60",(25 evaluaciones),24€,47€ en total
1,Habitación privada,Agradable habitación en pleno centro de Barcelona,La Font de la Guatlla,"1 huésped · 1 dormitorio · 1 cama · 1,5 baños ...",Wifi · Cocina · Aire acondicionado,10 dic. – 12 dic.,"4,57",(449 evaluaciones),29€,57€ en total
2,Habitación privada,"Room in the city center (Gothic,Rambla,Catedral)",Ciutat Vella,1 huésped · 1 dormitorio · 1 cama · 1 baño com...,Wifi · Cocina · Lavadora,24 dic. – 26 dic.,"4,48",(120 evaluaciones),25€,50€ en total
3,Habitación privada,Comfortable single room,Eixample,1 huésped · 1 dormitorio · 1 cama · 1 baño com...,Wifi · Cocina,31 dic. – 2 ene.,"4,57",(35 evaluaciones),37€,73€ en total
4,Alojamiento entero: piso,Outstanding aparment &Amazing location :),Ciutat Vella,2 huéspedes · 1 dormitorio · 1 cama · 1 baño,Wifi · Cocina · Aire acondicionado,17 dic. – 19 dic.,"4,77",(35 evaluaciones),85€,170€ en total


In [978]:
# Rename columns. 
airbnb.rename(columns= {
    "tipo": "type",
    "nombrealojamiento": "name",
    "zona": "district",
    "servicios":"services",
    "valoracion":"rating",
    "noche":"price_night"
}, inplace= True)

In [979]:
# Delete the ones we don't want. 
airbnb = airbnb.drop(["evaluaciones","total", "fechas"], axis=1)

In [980]:
#General view of the activities
unicos = airbnb["district"].unique()
for i in unicos:
    print(i)
len(unicos)

Ciutat Vella
La Font de la Guatlla
Eixample
El Camp d'en Grassot i Gràcia Nova
L'Hospitalet de Llobregat
Gràcia
L'Antiga Esquerra de l'Eixample
La Sagrada Familia
Sant Pere-Santa Caterina i la Ribera
Distrito de Sants-Montjuïc
Sant Marti
Les Corts
un apartamento con servicios
El Fort Pienc
El Poble Sec
Distrito de Horta-Guinardó
El Putxet i el Farró
La Teixonera
Pubilla Cases
Hostafrangs
El Prat de Llobregat
Distrito de San Andrés
complejo residencial
Sarrià-Sant Gervasi
Sant Adrià de Besòs
El Carmel


26

In [981]:
# Dict per distric 
dict_district = {
    r'[Cc]iutat|[Cc]aterina': 'Ciutat Vella',
    r'[Ss]agrada|[Pp]ienc|[Ee]ixample': 'Eixample',
    r'[Gg]uatlla|[Mm]ontju|[Po]ble Sec|[Ss]ants|[Hh]ostafran': 'Sants-Montjuic',
    r'[Cc]orts': 'Les Corts',
    r'[Gg]rassot|[Gg]ràcia': "Gracia",
    r'[Hh]orta|[Tt]eixoner|[Cc]armel': "Horta-Guinardo",
    r'[Aa]ndreu': "Sant Andreu",
    r'[Mm]arti': "Sant Marti",
    r"[Pp]utxet|[Ss]arria": "Sarria-Sant Gervasi"
 
}

In [982]:
# Function to categorize activities
def categorize_district(text, dict_district):
    if isinstance(text, str):  # Check if the value is a text string
        for pattern, category in dict_district.items():
            if re.search(pattern, text):
                return category
    return "Other"  # If it does not match any category, "Other" is returned.

# Clear invalid values in the "activity" column
airbnb["district"] =airbnb["district"].fillna("empty")  # Fill NaN values with an empty string
airbnb["district"] = airbnb["district"].astype(str)  # Convert all values to text strings

# Apply the function to categorize activities
airbnb["district"] = airbnb["district"].apply(lambda x: categorize_district(x, dict_district))

In [983]:
unicos = airbnb["district"].unique()
for i in unicos:
    print(i)
len(unicos)

Ciutat Vella
Sants-Montjuic
Eixample
Gracia
Other
Sant Marti
Les Corts
Horta-Guinardo
Sarria-Sant Gervasi


9

In [984]:
airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   type         300 non-null    object
 1   name         300 non-null    object
 2   district     300 non-null    object
 3   nhab         300 non-null    object
 4   services     300 non-null    object
 5   rating       287 non-null    object
 6   price_night  300 non-null    object
dtypes: object(7)
memory usage: 16.5+ KB


In [985]:
# Merge with distric df.
airbnb = pd.merge(airbnb, district, on='district')

In [986]:
airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   type         266 non-null    object
 1   name         266 non-null    object
 2   district     266 non-null    object
 3   nhab         266 non-null    object
 4   services     266 non-null    object
 5   rating       255 non-null    object
 6   price_night  266 non-null    object
 7   district_id  266 non-null    int8  
dtypes: int8(1), object(7)
memory usage: 14.9+ KB


In [987]:
# Drop the colun district because we only need the id. 
airbnb = airbnb.drop(["district"], axis=1)

In [988]:
airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   type         266 non-null    object
 1   name         266 non-null    object
 2   nhab         266 non-null    object
 3   services     266 non-null    object
 4   rating       255 non-null    object
 5   price_night  266 non-null    object
 6   district_id  266 non-null    int8  
dtypes: int8(1), object(6)
memory usage: 12.9+ KB


In [989]:
# We generated the uniques to created the id from them. 
airbnb_unique = airbnb["name"].unique()

In [990]:
# Make a df with the resolt. 
airbnb_id = pd.DataFrame(airbnb_unique, columns=["name"])

# Generated the index. 
airbnb_id["airbnb_id"] = pd.Series(airbnb_id["name"], dtype="category").cat.codes + 1

In [991]:
airbnb_id

,name,airbnb_id
0,"Habitacion centro Barcelona, Ciutat Vella room.",109
1,"Room in the city center (Gothic,Rambla,Catedral)",209
2,Outstanding aparment &Amazing location :),183
3,Habitación Individual con baño compartido,120
4,Mundi,173
...,...,...
259,"El Faro, habitacion privada 3",93
260,2 bed in single room Near Sagrada Familia,3
261,Sunny room for 2 persons near Park Guell Barce...,236
262,"Luxury loft new and fully furnished, top ameni...",168


In [992]:
# Merge the result with the airbnb df.
airbnb = pd.merge(airbnb, airbnb_id, on='name')

In [993]:
index_drop = [58,82,121]
airbnb = airbnb.drop(index_drop)

In [994]:
airbnb['price_night'] = airbnb['price_night'].str.replace('€', '')

In [995]:
airbnb.head()

,type,name,nhab,services,rating,price_night,district_id,airbnb_id
0,Habitación privada,"Habitacion centro Barcelona, Ciutat Vella room.",2 huéspedes · 1 dormitorio · 1 cama · 1 baño c...,Wifi · Cocina,"4,60",24,1,109
1,Habitación privada,"Room in the city center (Gothic,Rambla,Catedral)",1 huésped · 1 dormitorio · 1 cama · 1 baño com...,Wifi · Cocina · Lavadora,"4,48",25,1,209
2,Alojamiento entero: piso,Outstanding aparment &Amazing location :),2 huéspedes · 1 dormitorio · 1 cama · 1 baño,Wifi · Cocina · Aire acondicionado,"4,77",85,1,183
3,Habitación de hostal,Habitación Individual con baño compartido,"1 huésped · 1 dormitorio · 1 cama · 1,5 baños ...",Wifi · Aire acondicionado,"4,73",44,1,120
4,Habitación privada,Mundi,"1 huésped · 1 dormitorio · 1 cama · 1,5 baños ...",Wifi · Cocina · Lavadora,"4,56",32,1,173


In [996]:
airbnb['type'] = airbnb['type'].astype(str)
airbnb['name'] = airbnb['name'].astype(str)
airbnb['nhab'] = airbnb['nhab'].astype(str)
airbnb['services'] = airbnb['services'].astype(str)
airbnb['rating'] = airbnb['rating'].str.replace(',', '.').astype(float)
airbnb['price_night'] = airbnb['price_night'].astype(int)

In [997]:
airbnb.head(5)

,type,name,nhab,services,rating,price_night,district_id,airbnb_id
0,Habitación privada,"Habitacion centro Barcelona, Ciutat Vella room.",2 huéspedes · 1 dormitorio · 1 cama · 1 baño c...,Wifi · Cocina,4.60,24,1,109
1,Habitación privada,"Room in the city center (Gothic,Rambla,Catedral)",1 huésped · 1 dormitorio · 1 cama · 1 baño com...,Wifi · Cocina · Lavadora,4.48,25,1,209
2,Alojamiento entero: piso,Outstanding aparment &Amazing location :),2 huéspedes · 1 dormitorio · 1 cama · 1 baño,Wifi · Cocina · Aire acondicionado,4.77,85,1,183
3,Habitación de hostal,Habitación Individual con baño compartido,"1 huésped · 1 dormitorio · 1 cama · 1,5 baños ...",Wifi · Aire acondicionado,4.73,44,1,120
4,Habitación privada,Mundi,"1 huésped · 1 dormitorio · 1 cama · 1,5 baños ...",Wifi · Cocina · Lavadora,4.56,32,1,173


In [998]:
# I couldn't import en mysql because the comas so we intall this library to drop them.

In [999]:
pip install unidecode

Note: you may need to restart the kernel to use updated packages.


In [1000]:
from unidecode import unidecode

In [1001]:
airbnb['name'] = airbnb['name'].apply(lambda x: unidecode(str(x)))
airbnb['type'] = airbnb['type'].apply(lambda x: unidecode(str(x)))
airbnb['nhab'] = airbnb['nhab'].apply(lambda x: unidecode(str(x)))
airbnb['services'] = airbnb['services'].apply(lambda x: unidecode(str(x)))

In [1002]:
airbnb.to_csv('airbnb.csv', index=True)

In [1003]:
# Save the airbnb df. 
#airbnb.to_csv('airbnb.csv', index=True)

## POPULATION DATASET

In [855]:
# Import population DataFrame.
population = pd.read_csv("/Users/roraimachavez/Downloads/7.IRONHACK/Mini projects/sql/csv/bcn_dataset2015_2019.csv")

In [856]:
population.shape

(365, 38)

In [857]:
# We created a variable with the columns we need.
columnas_population = ['Unnamed: 0', 'year','neighbourhood', 'population','% spaniards','% strangers']

# Created the df from them. 
popu = population[columnas_population].copy()

In [858]:
# Filter.
popu = popu.loc[(popu['year']== 2019)] 

In [859]:
# Change to lower case. 
popu.columns = popu.columns.str.lower()

In [860]:
# Rename column. 
popu.rename(columns = {'unnamed: 0':'population_id'}, inplace = True) 

In [861]:
popu.head()

,population_id,year,neighbourhood,population,% spaniards,% strangers
292,292,2019,Baró de Viver,2625.0,85.1,14.9
293,293,2019,Can Baró,9331.0,82.1,17.9
294,294,2019,Can Peguera,2234.0,88.4,11.6
295,295,2019,Canyelles,6869.0,93.3,6.7
296,296,2019,Ciutat Meridiana,11091.0,67.2,32.8


In [862]:
# We created a df with district and neighbourhood df because we need the information of both of them. 
zone = pd.merge(district, neighbourhood, on='district_id')

In [863]:
zone.shape

(68, 4)

In [864]:
# We delete duplicates. 
zone.drop_duplicates(inplace = True)

In [865]:
zone.shape

(68, 4)

In [866]:
zone.head()

,district,district_id,neighbourhood,neighbourhood_id
0,Ciutat Vella,1,el Raval,43
1,Ciutat Vella,1,Gothic Quarter,6
2,Ciutat Vella,1,la Barceloneta,46
3,Ciutat Vella,1,"Sant Pere, Santa Caterina i la Ribera",21
4,Eixample,2,Fort Pienc,5


In [867]:
# We merge with the neighbourhood df because we need the id. 
popu = pd.merge(zone, popu, on='neighbourhood')

In [868]:
popu.head()

,district,district_id,neighbourhood,neighbourhood_id,population_id,year,population,% spaniards,% strangers
0,Ciutat Vella,1,el Raval,43,337,2019,48263.0,47.1,52.9
1,Ciutat Vella,1,la Barceloneta,46,340,2019,15112.0,59.5,40.5
2,Eixample,2,la Dreta de l'Eixample,48,343,2019,44325.0,74.6,25.4
3,Eixample,2,l'Antiga Esquerra de l'Eixample,45,339,2019,43228.0,74.4,25.6
4,Eixample,2,la Nova Esquerra de l'Eixample,55,350,2019,58621.0,78.9,21.1


In [869]:
# Here we make 2 new columns with the respective columns without % and make the result round to 1 decimal. 
popu['num_spaniards'] = round(popu['population'] * (popu['% spaniards'] / 100),1)
popu['num_strangers'] = round(popu['population'] * (popu['% strangers'] / 100),1)


# Drop the columns with %.
popu.drop(columns=['% spaniards', '% strangers'], inplace=True)

# And rename the new ones. 
popu.rename(columns={'num_spaniards': 'spaniards', 'num_strangers': 'strangers'}, inplace=True)


In [870]:
popu.head()

,district,district_id,neighbourhood,neighbourhood_id,population_id,year,population,spaniards,strangers
0,Ciutat Vella,1,el Raval,43,337,2019,48263.0,22731.9,25531.1
1,Ciutat Vella,1,la Barceloneta,46,340,2019,15112.0,8991.6,6120.4
2,Eixample,2,la Dreta de l'Eixample,48,343,2019,44325.0,33066.4,11258.6
3,Eixample,2,l'Antiga Esquerra de l'Eixample,45,339,2019,43228.0,32161.6,11066.4
4,Eixample,2,la Nova Esquerra de l'Eixample,55,350,2019,58621.0,46252.0,12369.0


In [871]:
# Only leave in the one with want. 
popu.drop(columns=['neighbourhood', 'district','district_id','year'], inplace=True)

In [872]:
popu.head()

,neighbourhood_id,population_id,population,spaniards,strangers
0,43,337,48263.0,22731.9,25531.1
1,46,340,15112.0,8991.6,6120.4
2,48,343,44325.0,33066.4,11258.6
3,45,339,43228.0,32161.6,11066.4
4,55,350,58621.0,46252.0,12369.0


In [873]:
# Save the popu df. 
#popu.to_csv('population.csv', index=True)

### BIN

In [874]:
neighbourhood.drop_duplicates(inplace=True)

In [875]:
neighbourhood.head()

,district_id,neighbourhood,neighbourhood_id
0,1,el Raval,43
4,1,Gothic Quarter,6
8,1,la Barceloneta,46
12,1,"Sant Pere, Santa Caterina i la Ribera",21
16,2,Fort Pienc,5


In [876]:
# Save neighbourhood df. 
# neighbourhood.to_csv('neighbourhood.csv', index=True)